# 🔍 Refining Keyword Search Results with Padding Control

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Keyword_Search_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 💬 Overview
--- 


In this tutorial, let's explore the enhanced search features, where accuracy meets adaptability. Easily discover the details of your video collection as we unlock the potential for effortless content discovery. This is an extension of the [Keyword Search tutorial](https://coda.io/d/_dnIYgjBK4eB/_suPo4) where you can refine the output with the flexibility to set the padding duration, ensuring thorough coverage around the keyword occurrence.

![](https://raw.githubusercontent.com/video-db/videodb-cookbook-assets/main/images/examples/Keyword_Search_2.png)

![](https://raw.githubusercontent.com/video-db/videodb-cookbook/main/images/Intro_Outro_Inline/image.png)

## Setup
---

### 📦  Installing packages 

In [ ]:
%pip install videodb

### 🔑 API Keys
Before proceeding, ensure access to [VideoDB](https://videodb.io) API key. 

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [1]:
import os

os.environ["VIDEO_DB_API_KEY"] = ""

## Steps
---

### 🌐 Step 1: Connect to VideoDB
Begin by establishing a connection to VideoDB using your API key

In [2]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()
coll = conn.get_collection()

### 🎥 Step 2: Upload Videos

Upload the video to VideoDB collection. You can upload the video asset from your local device or from a YouTube URL to upload the video from its source.

In [3]:
video = coll.upload(url="https://www.youtube.com/watch?v=Z2VBKerS63A")
video.play()

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/f93968ee-59ef-4a13-92ce-c8f703b37e46.m3u8'

### 🔊  Step 3: Index Spoken Words

Index the spoken words in your video to enable  keyword search.


In [4]:
video.index_spoken_words()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:58<00:00,  1.71it/s]


### 🔎  Step 4: Search for any keyword

Utilize the keyword search by using `Video.search()` method with following parameters.

* pass search query in `query` parameter
* pass `SearchType.keyword` in `search_type`

In [10]:
from videodb import SearchType

# Search Inside Video
results = video.search(query="generative", search_type=SearchType.keyword)

We can compile our search results into a single video.
Additionally, we will add some padding (both left and right) to give a some context to our each results shots.

Create a new timeline using `Timeline()`.  
Create VideoAsset of each result shot using `VideoAsset()` with following parameters  
- `asset_id` : Video Id of the Video to which result belongs to
- `start` : We will take result shot's `start` time with respect to base VideoAsset and to add some left padding, we will subtract `padding` duration
- `end` : We will take result shot's `end` time with respect to base VideoAsset and to add some right padding, we will add `padding` duration

Finally, we will add the asset in our Timeline by using `Timeline.add_inline()` 

In [13]:
from videodb.timeline import Timeline
from videodb.asset import VideoAsset

timeline = Timeline(conn)

# Add padding
padding = 0.4

# Compile Video
for shot in results.shots:
    asset = VideoAsset(
        asset_id=shot.video_id, start=shot.start - padding, end=shot.end + padding
    )
    timeline.add_inline(asset)

### 👀 Step 5: Preview and Share
Preview your video with a compilation of keywords you have searched for to ensure it aligns with what you were expecting. Share your findings with others to highlight the relevance and impact of the identified keywords within the video content.

In [11]:
from videodb import play_stream

stream = timeline.generate_stream()
play_stream(stream)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/91250123-61eb-43a4-99b3-9800666d7b8f.m3u8'

## 🎉 Conclusion
---
With VideoDB, you can harness the power of Keyword Search and refine your results with padding control. Simply by connecting to VideoDB, and utilizing the search function, you can easily discover relevant content within your video collection. With the added flexibility of setting padding duration, you ensure thorough coverage around keyword occurrences. Keep exploring and innovating with VideoDB to unlock even more possibilities for your projects!


For more such explorations, refer to the [VideoDB Docs](https://docs.videodb.io/) and join the VideoDB community on [GitHub](https://github.com/video-db) or [Discord](https://discord.com/invite/py9P639jGz) for support and collaboration.